# Task1A

In [ ]:
import sys
import subprocess
from PyQt5.QtCore import pyqtSlot, QTimer
from PyQt5.QtWidgets import QDialog, QApplication, QFileDialog
from PyQt5.uic import loadUi
from PyQt5.QtGui import QImage, QPixmap
from datetime import datetime
import json
import pandas as pd
import csv
from datetime import timedelta

data = {}
data['demo'] = []

#Reading csv for data stored by Demo GUI:
def update_row(data_dynamic_list, skill, demo_cnt, accuracy, diff, asset_updated_cnt, frames, days, feedback, iteration):
    data_dynamic_list.extend([skill, demo_cnt, accuracy, diff, asset_updated_cnt, frames, days, feedback, iteration])
    return data_dynamic_list

# Function to insert row in the dataframe 
def Insert_row(row_number, df, row_value): 
    # Slice the upper half of the dataframe 
    df1 = df[0:row_number] 
    # Store the result of lower half of the dataframe 
    df2 = df[row_number:] 
    # Insert the row in the upper half dataframe 
    df1.loc[row_number]=row_value
    # Concat the two dataframes 
    df_result = pd.concat([df1, df2])
    # Reassign the index labels 
    df_result.index = [*range(df_result.shape[0])]
    # Return the updated dataframe 
    return df_result

def write_csv(self, data_dynamic_list, skillname, feedback):
    df = pd.read_csv('data_dynamic.csv')

    if(feedback == 'NOT OK' or feedback == 'CANT TELL' and skillname != 'bumping'):
        row_num = self.learnable_count
        self.learnable_count += 1
        self.learnt_count += 1
        self.extrapolated_skills_count += 1
    elif(feedback == 'OK' and skillname != 'bumping'):
        row_num = self.learnt_count
        self.learnt_count += 1
        self.extrapolated_skills_count += 1
    elif(skillname == 'bumping'):
        row_num = self.extrapolated_skills_count
        self.extrapolated_skills_count += 1
    
    df_new = Insert_row(row_num, df, data_dynamic_list)
    df_new.to_csv('data_dynamic.csv', index=False)
    
def read_txt():
    with open('out.txt', 'r') as output:
        accuracy = output.read()
    return accuracy

def write_json(feedback, count, name, skill):
    data['demo'].append({
    'name': name,
    'datetime': str(datetime.now()),
    'skill' : skill,
    'feedback': feedback
    })
    #writing data in JSON
    with open('data_dict.json', 'w') as json_file:
        json.dump(data, json_file)
        print(data)
    count +=1
    text = str(count) + " - " + str(datetime.now().strftime("%x %X")) + " " + name + " " + skill
    print(text)
    return text, count
    
class opencvgui(QDialog):

    def __init__(self):
        super(opencvgui, self).__init__()
        loadUi('Navigator.ui', self)
        self.count = 0
        self.feedback = ''
        self.assets = []
        self.data_dynamic_list =['']
        #counter variables for number of demos:
        self.red_cnt = 0
        self.yellow_cnt = 0
        self.brown_cnt = 0
        self.crash_cnt = 0
        self.asset_updated_cnt = 0
        self.accuracy = 0
        #csv row count
        self.learnable_count = 1
        self.learnt_count = 2
        self.extrapolated_skills_count = 3
        #buttons
        self.RunBtn.setEnabled(True)
        self.NotOKBtn.setEnabled(True)
        self.CantTellBtn.setEnabled(True)
        self.OKBtn.setEnabled(True)
        self.exitBtn.setEnabled(True)
        #comboBox
        self.skillcomboBox.setEnabled(True)
        self.ItCBox.setEnabled(True)
        self.FCBox.setEnabled(True)
        #TextBox
        self.nameBox.setEnabled(True)
        self.DictionaryBox.setEnabled(True)
        #setting values from the click of run button:
        self.skill = None
        self.iteration = None
        self.frames = None
        self.feedback = None
        self.demo_cnt = None
        self.name = None
        self.a = None
        self.b = None
        self.c = None
        self.now = None
        self.then = None
        self.diff = None
        self.days = "0 days"
        #Mapping button clicks with proper functions:
        self.RunBtn.clicked.connect(self.runSkill)
        self.NotOKBtn.clicked.connect(self.NOKBtn)
        self.CantTellBtn.clicked.connect(self.CTBtn)
        self.OKBtn.clicked.connect(self.OkBtn)
        self.exitBtn.clicked.connect(sys.exit)
    
    @pyqtSlot()
    def runSkill(self):
        #call skill BHIRL to play
        self.now = datetime.now()
        # Getting skill name and adding it to a list to keep a track of unique skills(assets) updated
        self.skill = str(self.skillcomboBox.currentText())
        #updating count for unique assets updated
        if(self.skill not in self.assets):
            self.asset_updated_cnt += 1
            #time details:
            self.a = datetime.now()
        #time details:
        elif(self.skill in self.assets):
            #time details:
            self.b = datetime.now()
            self.c = self.b - self.a
            # Days since last update
            self.days = str(self.c.days) + " days"

        self.assets.append(self.skill)
        #Updating the demos count based on the skill name:
        if(self.skill == "red"):
            self.red_cnt += 1
            self.demo_cnt = self.red_cnt
        elif(self.skill == "yellow"):
            self.yellow_cnt += 1
            self.demo_cnt = self.yellow_cnt
        elif(self.skill == "brown"):
            self.brown_cnt += 1
            self.demo_cnt = self.brown_cnt
        elif(self.skill == "bumping"):
            self.crash_cnt += 1
            self.demo_cnt = self.crash_cnt
           
        self.iteration = str(self.ItCBox.currentText())
        self.frames = str(self.FCBox.currentText())
        subprocess.call(['bash', 'start.sh', self.skill, self.iteration, self.frames])    
        print("give feedback")
        self.accuracy = read_txt()
        # Time taking for one run in minutes
        self.then = datetime.now()
        self.diff = self.then - self.now
        self.diff = str(round(self.diff / timedelta(minutes=1), 4)) + " min"
        print(self.diff)

    def NOKBtn(self):
        self.feedback = 'NOT OK'
        self.name = self.nameBox.document().toPlainText()
        self.skill = str(self.skillcomboBox.currentText())
        text, cnt = write_json(self.feedback, self.count, self.name, self.skill)
        self.count = cnt
        self.DictionaryBox.append(text + " : " + self.feedback)
        #building rows and writing rows
        self.data_dynamic_list = update_row(self.data_dynamic_list, self.skill, self.demo_cnt, self.accuracy, self.diff, \
                   self.asset_updated_cnt, self.frames, self.days, self.feedback, self.iteration)
        write_csv(self, self.data_dynamic_list, self.skill, self.feedback)
        # setting the row list as empty for next demo
        self.data_dynamic_list = ['']

    def CTBtn(self):
        self.feedback = 'CANT TELL'
        self.name = self.nameBox.document().toPlainText()
        self.skill = str(self.skillcomboBox.currentText())
        text, cnt = write_json(self.feedback, self.count, self.name, self.skill)
        self.count = cnt
        self.DictionaryBox.append(text + " : " + self.feedback)
        #building rows and writing rows
        self.data_dynamic_list = update_row(self.data_dynamic_list, self.skill, self.demo_cnt, self.accuracy, self.diff, \
                   self.asset_updated_cnt, self.frames, self.days, self.feedback, self.iteration)
        write_csv(self, self.data_dynamic_list, self.skill, self.feedback)
        # setting the row list as empty for next demo
        self.data_dynamic_list = ['']

    def OkBtn(self):
        self.feedback = 'OK'
        self.name = self.nameBox.document().toPlainText()
        self.skill = str(self.skillcomboBox.currentText())
        text, cnt = write_json(self.feedback, self.count, self.name, self.skill)
        self.count = cnt
        self.DictionaryBox.append(text + " : " + self.feedback)
        #building rows and writing rows
        self.data_dynamic_list = update_row(self.data_dynamic_list, self.skill, self.demo_cnt, self.accuracy, self.diff, \
                   self.asset_updated_cnt, self.frames, self.days, self.feedback, self.iteration)
        write_csv(self, self.data_dynamic_list, self.skill, self.feedback)
        # setting the row list as empty for next demo
        self.data_dynamic_list = ['']

#runnning Qt App
app = QApplication(sys.argv)
window = opencvgui()
window.setWindowTitle("Demonstrations Dictionary")
window.show()
sys.exit(app.exec_())

give feedback
acc 
0.0071 min
{'demo': [{'name': 'nb', 'datetime': '2019-10-13 01:52:17.168074', 'skill': 'red', 'feedback': 'NOT OK'}]}
1 - 10/13/19 01:52:17 nb red
                     Type  Name  Demos  Accuracy  Time  Assets<br>Updated  \
0               Learnable   NaN    NaN       NaN   NaN                NaN   
1                  Learnt   NaN    NaN       NaN   NaN                NaN   
2  Extrapolated<br>Skills   NaN    NaN       NaN   NaN                NaN   

   Total<br>Episodes  Submitted  Latest<br>Feedback  Epochs  
0                NaN        NaN                 NaN     NaN  
1                NaN        NaN                 NaN     NaN  
2                NaN        NaN                 NaN     NaN  
['', 'red', 1, '', '0.0071 min', 1, '1000', '0 days', 'NOT OK', '1']


/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


give feedback
acc 
0.0031 min
{'demo': [{'name': 'nb', 'datetime': '2019-10-13 01:52:17.168074', 'skill': 'red', 'feedback': 'NOT OK'}, {'name': 'nb', 'datetime': '2019-10-13 01:52:20.886412', 'skill': 'red', 'feedback': 'CANT TELL'}]}
2 - 10/13/19 01:52:20 nb red
                     Type Name  Demos  Accuracy        Time  \
0               Learnable  NaN    NaN       NaN         NaN   
1                     NaN  red    1.0       NaN  0.0071 min   
2                  Learnt  NaN    NaN       NaN         NaN   
3  Extrapolated<br>Skills  NaN    NaN       NaN         NaN   

   Assets<br>Updated  Total<br>Episodes Submitted Latest<br>Feedback  Epochs  
0                NaN                NaN       NaN                NaN     NaN  
1                1.0             1000.0    0 days             NOT OK     1.0  
2                NaN                NaN       NaN                NaN     NaN  
3                NaN                NaN       NaN                NaN     NaN  
['', 'red', 2, '', '0.00

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


give feedback
acc 
0.0034 min
{'demo': [{'name': 'nb', 'datetime': '2019-10-13 01:52:17.168074', 'skill': 'red', 'feedback': 'NOT OK'}, {'name': 'nb', 'datetime': '2019-10-13 01:52:20.886412', 'skill': 'red', 'feedback': 'CANT TELL'}, {'name': 'nb', 'datetime': '2019-10-13 01:52:24.368899', 'skill': 'red', 'feedback': 'OK'}]}
3 - 10/13/19 01:52:24 nb red
['', 'red', 3, '', '0.0034 min', 1, '1000', '0 days', 'OK', '1']
                     Type Name  Demos  Accuracy        Time  \
0               Learnable  NaN    NaN       NaN         NaN   
1                     NaN  red    1.0       NaN  0.0071 min   
2                     NaN  red    2.0       NaN  0.0031 min   
3                  Learnt  NaN    NaN       NaN         NaN   
4  Extrapolated<br>Skills  NaN    NaN       NaN         NaN   

   Assets<br>Updated  Total<br>Episodes Submitted Latest<br>Feedback  Epochs  
0                NaN                NaN       NaN                NaN     NaN  
1                1.0             1000.0  

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


give feedback
acc 
0.0032 min
{'demo': [{'name': 'nb', 'datetime': '2019-10-13 01:52:17.168074', 'skill': 'red', 'feedback': 'NOT OK'}, {'name': 'nb', 'datetime': '2019-10-13 01:52:20.886412', 'skill': 'red', 'feedback': 'CANT TELL'}, {'name': 'nb', 'datetime': '2019-10-13 01:52:24.368899', 'skill': 'red', 'feedback': 'OK'}, {'name': 'nb', 'datetime': '2019-10-13 01:52:30.550303', 'skill': 'bumping', 'feedback': 'OK'}, {'name': 'nb', 'datetime': '2019-10-13 01:52:36.015701', 'skill': 'yellow', 'feedback': 'CANT TELL'}]}
5 - 10/13/19 01:52:36 nb yellow
                     Type     Name  Demos  Accuracy        Time  \
0               Learnable      NaN    NaN       NaN         NaN   
1                     NaN      red    1.0       NaN  0.0071 min   
2                     NaN      red    2.0       NaN  0.0031 min   
3                  Learnt      NaN    NaN       NaN         NaN   
4                     NaN      red    3.0       NaN  0.0034 min   
5  Extrapolated<br>Skills      NaN    Na

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


give feedback
acc 
0.0031 min
{'demo': [{'name': 'nb', 'datetime': '2019-10-13 01:52:17.168074', 'skill': 'red', 'feedback': 'NOT OK'}, {'name': 'nb', 'datetime': '2019-10-13 01:52:20.886412', 'skill': 'red', 'feedback': 'CANT TELL'}, {'name': 'nb', 'datetime': '2019-10-13 01:52:24.368899', 'skill': 'red', 'feedback': 'OK'}, {'name': 'nb', 'datetime': '2019-10-13 01:52:30.550303', 'skill': 'bumping', 'feedback': 'OK'}, {'name': 'nb', 'datetime': '2019-10-13 01:52:36.015701', 'skill': 'yellow', 'feedback': 'CANT TELL'}, {'name': 'nb', 'datetime': '2019-10-13 01:52:41.720447', 'skill': 'yellow', 'feedback': 'OK'}]}
6 - 10/13/19 01:52:41 nb yellow
['', 'yellow', 2, '', '0.0031 min', 3, '1000', '0 days', 'OK', '2']
                     Type     Name  Demos  Accuracy        Time  \
0               Learnable      NaN    NaN       NaN         NaN   
1                     NaN      red    1.0       NaN  0.0071 min   
2                     NaN      red    2.0       NaN  0.0031 min   
3          

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


# Task1B

In [ ]:
import plotly.graph_objects as go
import plotly.figure_factory as ff
import pandas as pd
import re
import csv
import numpy as np

#Reading csv for data stored by Demo GUI:
with open('data.csv', 'r') as f:
    reader = csv.reader(f)
    table_data = list(reader)

# Initialize a figure with ff.create_table(table_data)
fig = ff.create_table(table_data, height_constant=60)

# Fetch data to plot Epochs Vs Accuracy:
df = pd.read_csv("data.csv")
epoch_list = []
for i, row in enumerate(df["Epochs"]):
    epoch_list.append(int(row))

max_epoch = max(epoch_list)
epochs = np.arange(max_epoch+1)
red_acc = ['0']
yellow_acc = ['0']
brown_acc = ['0']
crash_acc = ['0']

for i, row in enumerate(df["Name"]):
    if(row == "red"):
        red_acc.append(df["Accuracy"][i])
    elif(row == "yellow"):
        yellow_acc.append(df["Accuracy"][i])
    elif(row == "brown"):
        brown_acc.append(df["Accuracy"][i])
    elif(row == "bumping"):
        crash_acc.append(df["Accuracy"][i])

#Generating traces of plots for all skills:
trace1 = go.Scatter(x=epochs, y=red_acc,
                    marker=dict(color='#FF0000'),
                    name= 'Red Skill',
                    xaxis='x2', yaxis='y2')
trace2 = go.Scatter(x=epochs, y=yellow_acc,
                    marker=dict(color='#FFFF00'),
                    name= 'Yellow Skill',
                    xaxis='x2', yaxis='y2')

trace3 = go.Scatter(x=epochs, y=brown_acc,
                    marker=dict(color='#654321'),
                    name= 'Brown Skill',
                    xaxis='x2', yaxis='y2')

trace4 = go.Scatter(x=epochs, y=crash_acc,
                    marker=dict(color='#008000'),
                    name= 'Crash Skill',
                    xaxis='x2', yaxis='y2')
# Add trace data to figure
fig.add_traces([trace1, trace2, trace3, trace4])

# initialize xaxis2 and yaxis2
fig['layout']['xaxis2'] = {}
fig['layout']['yaxis2'] = {}

# Edit layout for subplots
fig.layout.yaxis.update({'domain': [0, .5]})
fig.layout.yaxis2.update({'domain': [.6, 1]})

# The graph's yaxis2 MUST BE anchored to the graph's xaxis2 and vice versa
fig.layout.yaxis2.update({'anchor': 'x2'})
fig.layout.xaxis2.update({'anchor': 'y2'})
fig.layout.yaxis2.update({'title': 'Accuracy'})
fig.layout.xaxis2.update({'title': 'Epochs'})

# Update the margins to add a title and see graph x-labels.
fig.layout.margin.update({'t':25, 'l':25})
fig.layout.update({'title': 'Learning Efficiency'})

# Update the height because adding a graph vertically will interact with
# the plot height calculated for the table
fig.layout.update({'height':800})

# Plot!
fig.show()